# Linear regression

In [1]:
import numpy as np
import math
import random

In [2]:
# data set
X = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

In [3]:
def cost(X, y, w, b):

    # defining local vars
    cost = 0
    loss = 0
    y_pred = 0
    m = len(X)

    for i in range(m):
        # making pred with the current weights
        y_pred = w*X[i] + b
        loss = (y_pred - y[i])**2
        cost += loss
        
    cost = cost /(2*m)
    return cost


In [4]:
def grad_calc(X,y,w,b):
    
    m = len(X)
    d_w = 0
    d_b = 0
    
    for i in range(m):
        
        pred = w*X[i]+b
        
        d_w += (y[i]-pred)*X[i]
        d_b += (y[i]-pred)
        
    d_w *= (-2/m) 
    d_b *= (-2/m)

    return d_w, d_b

In [5]:
def linearRegression(X, y, w, b):

    a = 0.01
    d_b = 0
    d_w = 0
    m = len(X)

    for i in range(100):
        if i % 10 == 0:
            print(cost(X, y, w, b))
        
        d_w, d_b = grad_calc(X, y, w, b)
        
        w += -a * d_w
        b += -a * d_b

    return w, b

In [6]:
# final run
w, b = linearRegression(X,y,0,0)
print(w,b)

77.0
0.007804279269779495
0.007174295455124458
0.006595165744224451
0.006062785045007175
0.005573379643135202
0.005123480449318258
0.004709898409106042
0.004329701897672176
0.003980195939356661
1.973457186117349 0.18478635542525906


### Logistic regression

In [7]:
X = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
y = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

In [8]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [9]:
def calc_cost(X,y,w,b):
    cost = 0
    loss = 0
    m = len(X)

    for i in range(m):
        y_pred = sigmoid(w*X[i] + b)
        loss = -(y[i] * np.log(y_pred) + (1 - y[i]) * np.log(1 - y_pred))
        cost += loss
        
        
    return cost /m

In [10]:
def calc_gradiant(X,y,w,b):
    d_w = 0
    d_b = 0
    m = len(X)
    
    for i in range(m):
        
        z = w*X[i]+b
        pred = sigmoid(z)
        
        d_w += (pred - y[i])*X[i]
        d_b += (pred-y[i])
        
    d_w = d_w/m    
    d_b = d_b/m
    
    return d_w,d_b 

In [11]:
def train_model(X,y,w,b,a):
    d_b = 0
    d_w = 0
    m = len(X)
    for i in range(100):
        if i % 10 == 0:
            print(calc_cost(X,y,w,b))
            
        d_w,db = calc_gradiant(X,y,w,b)
        w += -a*d_w
        b += -a*d_b
    return w,b 

In [12]:
train_model(X,y,1,2,0.01)

1.770210843041627
1.7178711594440137
1.6661184912221336
1.6150704821263466
1.564870883624968
1.5156945004030216
1.4677521379495206
1.4212947710642898
1.3766156346446485
1.3340483206439537


(0.3102370857034073, 2.0)

# Making a class that will help us with backprop

A neuron is basicly a function. and in order for us to be able to optimize it with backprop we need to know the children of each node in the function, and the operation which leads to each node. and in order to do that and some more cool operation, we are going to make a class called value(which is equivalent to the autograd engine in pytorch)

In [13]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

# Making a class for building neurons

In [14]:
# Making the neuron class
class Neuron:
  
  def __init__(self, nin):
    # generating nin weights and 1 random bias
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
     #x = [Value(xi) for xi in x]
    # looping through w and x and multiplying corresponding xs and ws(at the end we also add the bias)
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
      # running the linear output through an activation function
    out = act.tanh()

    return out

    # making a function that returns a list of the params in a neuron. this is necessary for backprop
  def parameters(self):
    return self.w + [self.b]

In [15]:
x = [5.0,7.0]
# defining number of features
n = Neuron(2)
n(x)

Value(data=-0.9930653510988443)

In [16]:
class Layer:

  # making nouts neurons with nin features
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]

  def __call__(self, x):
    # looping thruge the neurons, inputing x to them, and then appending the output to a list.
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

In [17]:
a = Layer(2,4)
print(a(x))

[Value(data=-0.9501112779653673), Value(data=-0.999127008660747), Value(data=0.40283348980863787), Value(data=0.9988921268628729)]


# Making a Neural Network(MLP)

In [18]:
class MLP:

  # nin num of input features, nouts a list that specifies the number of neurons in each layer of the network.
  def __init__(self, nin, nouts):
    sz = [nin] + nouts

    # Creating a list of Layer objects, each with the current number of inputs (sz[i]) and the next number of neurons (sz[i+1])
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):

    # looping througe each layer
    for layer in self.layers:
      # inputing x to the iterated layer, and assingnig its output to x .
      x = layer(x)
    return x

  # returning all the params. 
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [19]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=0.18633264655100948)

# Training the neural network

In [1]:
import os
print(os.getcwd())

C:\Users\yadga
